In [1]:
import json
import pandas as pd
import re
import random
import numpy as np
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

######## METRICS

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


# matplotlib
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline

# other
from datetime import datetime

In [2]:
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    return reviews

In [5]:
def prepare_dataset(filename):
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []

    #набираем список хороших/плохих отзывов, не более 75000 в каждом из списков
    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=75000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=75000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 150000:
                    break
    
    # перемешиваем список (внутри элементы одного класса)
    random.shuffle(reviews_lst_good)
    random.shuffle(reviews_lst_bad)  
    
    # создаём template для сохранения конечных отзывов
    reviewText_lst = []
    mark_lst = []
    
    
    # где будем уменьшать количество отзывов, чтобы их было равное количество
    if len (reviews_lst_bad) < len(reviews_lst_good):
        count_otz = len (reviews_lst_bad)
    else:
        count_otz = len (reviews_lst_good)
        
    
    # i -- это СЛОВАРЬ!        
    # обрезаем хороший и наполняем уже итоговый список
    for i in reviews_lst_good[:count_otz]:
        reviewText_lst.append (i['reviewText'])    
        mark_lst.append (1)
    print ('Good: ',len(reviews_lst_good[:count_otz]))

    # обрезаем плохие
    for i in reviews_lst_bad[:count_otz]:
        reviewText_lst.append (i['reviewText'])
        mark_lst.append (-1)
    print ('Bad: ',len(reviews_lst_bad[:count_otz]))

    return reviewText_lst, mark_lst
    
def classify_it(filename, reviewText_lst, mark_lst):  
    
    # выборки готовы к передачи в модель
    reviews_train = reviewText_lst
    y = mark_lst
    
    # очищаем отзывы от лишних символов
    reviews_train_clean = preprocess_reviews(reviews_train)
    
    
    # делим выборку на тестовую и обучающую
    X_train, X_val, y_train, y_val = train_test_split(
        reviews_train_clean, y, train_size = 0.75
    )  
    
    # столбец ответов 
    

    print ('Подбор параметра для: ', filename[:-5])
    print ('    ')
    best_c_dict = {} # модель сама ищет лучший c и его результаты подбирает в итоге
    c_list = []
    result_c_list = []      
   
    time_lst_fit = []
    time_frcst_lst = []
    result_time_fit_lst = []
    result_time_frcst_lst = []
    
    # определяем, где алгоритму необходимо искать С
    grid = [0.01,0.1,1,10]
    
    
    print ('Бинаризируем столбец признаков')
    # бинаризируем столбец признаков
    cv = TfidfVectorizer(min_df=10, decode_error='replace', encoding='utf-8')
    X = cv.fit_transform(X_train)
    X_test = cv.transform(X_val)
    
#    X = preprocessing.scale(X,with_mean=False)
#    X_test = preprocessing.scale(X_test,with_mean=False)
    
    print ('c подбираем')

    
    # итерируемся и подбираем параметр, используя Accuracy
    for c in grid:
        
        final_model = svm.SVC(kernel='linear', C=c, decision_function_shape='ovo')
        
        startTime = time.time()
        final_model.fit(X, y_train) ######
        resultTime = time.time() - startTime
        time_lst_fit.append (resultTime)
        
        startTime = time.time()
        y_pred = final_model.predict(X_test) #####
        resultTime = time.time() - startTime
        time_frcst_lst.append (resultTime)
        
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, y_pred)))
        
        c_list.append (c)
        result_c_list.append (accuracy_score(y_val, y_pred))
        best_c_dict [c] = accuracy_score(y_val, y_pred)
        
    
    # записываем результаты подбора коэффициента c
    result_с = pd.DataFrame()
    result_с['c'] = c_list
    result_с ['result_acc'] = result_c_list
    result_с['fit_time'] = time_lst_fit
    result_с['forecast_time'] = time_frcst_lst
    result_с.to_csv (filename[:-5]+'_podbor_c_svm.csv', index=False)
    ###########################################################################
    
    inverse = [(value, key) for key, value in best_c_dict.items()]
    print ('The best C is ', max(inverse)[1])
    
    
    ### строим предсказание с лучшим c ++ считаем потраченное время
    final_model_last = svm.SVC(kernel='linear', C=max(inverse)[1], decision_function_shape='ovo')
    final_model_last.fit(X, y_train)
    y_pred_last = final_model.predict(X_test)
    ################

    # смотрим на метрики 
    print ("Final Accuracy: %s" 
           % accuracy_score(y_val
                            , y_pred_last))
    
    print('confusion_matrix')
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(y_val, y_pred_last)
    print(confusion_matrix)
    
    print ('classification_report')
    print(classification_report(y_val, y_pred_last))


In [6]:
import os
import gc
import time


# G:\\Мой диск\\Work\\Experiment
#/Volumes/GoogleDrive/Мой диск/Work/Experiment

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb'and i[-3:] != 'csv' and i != '.ipynb_checkpoints': # and i == 'Health_and_Personal_Care_5.json':
        print (i[:-5])
        print ('   ')
        x,y = prepare_dataset ('G:\\Мой диск\\Work\\Experiment\\' + i)
        
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i,x,y)
        
        gc.collect()
        

Health_and_Personal_Care_5
   
Good:  33300
Bad:  33300
Подбор параметра для:  G:\Мой диск\Work\Experiment\Health_and_Personal_Care_5
    
Бинаризируем столбец признаков
c подбираем
Accuracy for C=0.01: 0.8062462462462463
Accuracy for C=0.1: 0.8561561561561561
Accuracy for C=1: 0.8652252252252253
Accuracy for C=10: 0.8506306306306306
The best C is  1
Final Accuracy: 0.8506306306306306
confusion_matrix
[[7102 1243]
 [1244 7061]]
classification_report
              precision    recall  f1-score   support

          -1       0.85      0.85      0.85      8345
           1       0.85      0.85      0.85      8305

   micro avg       0.85      0.85      0.85     16650
   macro avg       0.85      0.85      0.85     16650
weighted avg       0.85      0.85      0.85     16650

Movies_and_TV_5
   
Good:  74999
Bad:  74999
Подбор параметра для:  G:\Мой диск\Work\Experiment\Movies_and_TV_5
    
Бинаризируем столбец признаков
c подбираем
Accuracy for C=0.01: 0.85248
Accuracy for C=0.1: 0.89666666

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 759: character maps to <undefined>

In [ ]:
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ": Main script Started")

print (time.sleep(5))


print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ": Main script End")



In [23]:
df = pd.read_csv ('G:\\Мой диск\\Work\\Experiment\\'+'Movies_and_TV_5_podbor_c_svm.csv', engine = 'python')
df

,c,result_acc,fit_time,forecast_time
0,0.01,0.485714,0 days 00:00:00.007974000,0 days 00:00:00.001071000
1,0.10,0.828571,0 days 00:00:00.004074000,0 days 00:00:00.000996000
2,1.00,0.914286,0 days 00:00:00.002997000,0 days 00:00:00.000000000
3,10.00,0.942857,0 days 00:00:00.002991000,0 days 00:00:00.000995000
4,100.00,0.942857,0 days 00:00:00.004002000,0 days 00:00:00.000000000


In [27]:
# pd.to_datetime(df['fit_time'])-pd.to_datetime(df['forecast_time'])

pd.to_datetime(df['fit_time'][0][7:],)

Timestamp('2019-05-24 00:00:00.007974')

Comments

https://habr.com/ru/company/ods/blog/323890/#1-lineynaya-regressiya

https://docplayer.ru/28834116-Otchet-po-zadaniyu-4-issledovanie-modeli-logistic-regression.html

https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_(%D0%BA%D1%83%D1%80%D1%81_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B9,_%D0%9A.%D0%92.%D0%92%D0%BE%D1%80%D0%BE%D0%BD%D1%86%D0%BE%D0%B2)#.D0.9E.D1.81.D0.BD.D0.BE.D0.B2.D0.BD.D1.8B.D0.B5_.D0.BF.D0.BE.D0.BD.D1.8F.D1.82.D0.B8.D1.8F_.D0.B8_.D0.BF.D1.80.D0.B8.D0.BC.D0.B5.D1.80.D1.8B_.D0.BF.D1.80.D0.B8.D0.BA.D0.BB.D0.B0.D0.B4.D0.BD.D1.8B.D1.85_.D0.B7.D0.B0.D0.B4.D0.B0.D1.87



ROC:
https://basegroup.ru/community/articles/logistic

https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8